In [6]:
!pip install duckdb ipython-sql pandas

  Obtaining dependency information for ipython-sql from https://files.pythonhosted.org/packages/30/8f/9e50fa53ffc371483f9d1b90c1175b706d28a2e978e90a8894035af01905/ipython_sql-0.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for prettytable from https://files.pythonhosted.org/packages/73/19/4bb9530512432774fdd7cb7c020851d4decbb811d95f86fd4f6a870a6d3e/prettytable-3.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for sqlparse from https://files.pythonhosted.org/packages/7a/13/5f6654c9d915077fae255686ca6fa42095b62b7337e3e1aa9e82caa6f43a/sqlparse-0.5.2-py3-none-any.whl.metadata
  Obtaining dependency information for ipython-genutils from https://files.pythonhosted.org/packages/fa/bc/9bd3b5c2b4774d5f33b2d544f1460be9df7df2fe42f352135381c347c69a/ipython_genutils-0.2.0-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run

In [7]:
import duckdb

# Connect to the DuckDB database
conn = duckdb.connect('../hmp_microbiome.duckdb')

In [8]:
# Execute a query and fetch results as a Pandas DataFrame
df = conn.execute("SELECT * FROM organisms LIMIT 5").fetchdf()
print(df)

   organism_id          accession                        organism_name  \
0            1  NZ_ACGB01000060.1              Acidaminococcus sp. D21   
1            2      NZ_GG770410.1  Achromobacter piechaudii ATCC 43553   
2            3      NZ_GL635869.1       Achromobacter xylosoxidans C54   
3            5      NZ_GG704511.1            Acinetobacter sp. RUH2624   
4            4      NZ_GG665950.1         Acinetobacter sp. ATCC 27244   

                                            taxonomy  genome_size  \
0  Bacteria;Firmicutes;Negativicutes;Selenomonada...         8820   
1  Bacteria;Proteobacteria;Betaproteobacteria;Bur...         2687   
2  Bacteria;Proteobacteria;Betaproteobacteria;Bur...         1515   
3  Bacteria;Proteobacteria;Gammaproteobacteria;Ps...        98692   
4  Bacteria;Proteobacteria;Gammaproteobacteria;Ps...       357640   

          processing_date                                         source_url  
0 2024-10-24 08:04:29.982  https://ftp.ncbi.nih.gov/genomes/H

In [9]:
%load_ext sql

In [12]:
%sql duckdb:///:hmp_microbiome.duckdb

Traceback (most recent call last):
  File "/home/shawn/projects/data-pipeline-microbiom-duckdb/venv/lib/python3.11/site-packages/sql/connection.py", line 45, in __init__
    engine = sqlalchemy.create_engine(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 2, in create_engine
  File "/home/shawn/projects/data-pipeline-microbiom-duckdb/venv/lib/python3.11/site-packages/sqlalchemy/util/deprecations.py", line 281, in warned
    return fn(*args, **kwargs)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^
  File "/home/shawn/projects/data-pipeline-microbiom-duckdb/venv/lib/python3.11/site-packages/sqlalchemy/engine/create.py", line 550, in create_engine
    entrypoint = u._get_entrypoint()
                 ^^^^^^^^^^^^^^^^^^^
  File "/home/shawn/projects/data-pipeline-microbiom-duckdb/venv/lib/python3.11/site-packages/sqlalchemy/engine/url.py", line 758, in _get_entrypoint
    cls = registry.load(name)
          ^^^^^^^^^^^^^^^^^^^
  File "/home/shawn/projects/dat

In [13]:
conn.execute("SHOW TABLES").fetchdf()

,name
0,gene_qualifiers
1,genes
2,organisms
3,processing_log
4,sequences


In [14]:
# Set the number of threads for parallel processing
conn.execute("PRAGMA threads=4;")

# Query to count genes per organism using parallel execution
query = """
SELECT organism_name, COUNT(gene_id) AS gene_count 
FROM organisms o 
JOIN genes g ON o.organism_id = g.organism_id 
GROUP BY organism_name;
"""
result = conn.execute(query).fetchdf()
print(result)

                             organism_name  gene_count
0          Dysgonomonas gadei ATCC BAA-286         628
1                Eggerthella sp. 1_3_56FAA           4
2       Enterobacter hormaechei ATCC 49162         407
3             Enterococcus faecalis TX1302          12
4             Enterococcus faecalis TX2137           2
..                                     ...         ...
885    Corynebacterium accolens ATCC 49726           4
886               Dermacoccus sp. Ellin185          12
887                    Desmospora sp. 8437           4
888           Desulfovibrio sp. 6_1_46AFAA           4
889  Corynebacterium genitalium ATCC 33030        4575

[890 rows x 2 columns]


In [15]:
# Export query results to a CSV file
query = """
COPY (SELECT * FROM organisms LIMIT 10) TO 'output.csv' WITH (FORMAT CSV, HEADER TRUE);
"""
conn.execute(query)
print("Query results exported to 'output.csv'")

Query results exported to 'output.csv'


In [17]:
conn.close()